In [3]:
# ============================
# RunPod version of your Step 5 pipeline
# (No Google Drive, paths changed to /workspace)
# ============================

# 0) (Optional) show GPU + environment
import torch, platform, os, sys, json, pandas as pd
print("Torch:", torch.__version__, "| CUDA:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
!nvidia-smi
!pip install "numpy<2" --force-reinstall

# 1) Set RunPod paths (replaces drive.mount + /content paths)
import os
ROOT = "/workspace/attention-collapse-llm"
STEP5 = f"{ROOT}/step5"
OUT   = f"{ROOT}/step5_outputs"
DATA  = f"{ROOT}/data"
os.makedirs(STEP5, exist_ok=True)
os.makedirs(OUT,   exist_ok=True)
os.makedirs(DATA,  exist_ok=True)
print("Folders ready:", ROOT)

# 2) Clean & install compatible versions (same as your Colab)
#    NOTE: The RunPod image already has torch; this matches your pins.
!pip -q uninstall -y transformers torchvision torchaudio > /dev/null 2>&1
!pip -q install --upgrade "torch==2.3.1" "torchvision==0.18.1" --index-url https://download.pytorch.org/whl/cu121
!pip -q install "transformer-lens==2.7.0" "einops==0.8.1" "transformers==4.41.2"
print("Installed torch/torchvision/transformer-lens/transformers")

# 3) Paths + config (unchanged except for ROOT)
import os, json, pandas as pd

paths = {
    "shortlist_csv": f"{OUT}/step5_head_shortlist.csv",
    "plan_csv":      f"{OUT}/step5_ablation_plan.csv",
    "examples_csv":  f"{STEP5}/merged_predictions_metrics.csv",
    "per_head_csv":  f"{DATA}/gpt2small_attn_metrics_per_head.csv",
    "qa_csv":        f"{STEP5}/Distractors - Sheet1.csv",
    "output_csv":    f"{OUT}/step5_ablation_results.csv",
    "config_json":   f"{STEP5}/step5_ablation_config.json",
}
print("Checking files:")
for k,p in paths.items():
    if k.endswith("_json"): 
        continue
    print(k, "->", os.path.exists(p), "|", p)

CFG = {
  "paths": {
    "shortlist_csv": paths["shortlist_csv"],
    "plan_csv":      paths["plan_csv"],
    "examples_csv":  paths["examples_csv"],
    "per_head_csv":  paths["per_head_csv"],
    "output_csv":    paths["output_csv"]
  }
}
with open(paths["config_json"], "w") as f:
    json.dump(CFG, f, indent=2)
print("Config written:", paths["config_json"])

# 4) Merge gold answers into examples (same logic)
import pandas as pd, numpy as np

qa = pd.read_csv(paths["qa_csv"])
assert "Question" in qa.columns and "Gold Text" in qa.columns, "QA sheet must have 'Question' and 'Gold Text'."

gold_map = qa.rename(columns={"Question":"question","Gold Text":"gold_answer"})[["question","gold_answer"]].copy()
gold_map["question"] = gold_map["question"].astype(str).str.strip()
gold_map["gold_answer"] = gold_map["gold_answer"].astype(str).str.strip()

ex = pd.read_csv(paths["examples_csv"])
ex["question"] = ex["question"].astype(str).str.strip()

# Drop the existing gold_answer column from ex if it exists to avoid merge conflicts
if "gold_answer" in ex.columns:
    ex = ex.drop(columns=["gold_answer"])

merged = ex.merge(gold_map, on="question", how="left")
merged.to_csv(paths["examples_csv"], index=False)

print("Merged gold answers into examples")
print("rows:", len(merged), " | missing gold:", merged["gold_answer"].isna().sum())
display(merged.head(3))

# 5) Write the runner to /workspace path (not /content)
runner_code = r'''
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Step 5 — Head-ablation paired evaluation with lightweight collapse metrics.
Outputs per-row before/after answers + metrics for external scoring.
"""

import os, json
os.environ["TRANSFORMERS_NO_TORCHVISION"] = "1"  # avoid torchvision import issues

import pandas as pd
import numpy as np
from pathlib import Path
from typing import Dict, Any, List, Tuple

# ------------------------------
# Load config & data
# ------------------------------
CFG = json.load(open("step5_ablation_config.json","r"))
SHORTLIST = Path(CFG["paths"]["shortlist_csv"])
PLAN      = Path(CFG["paths"]["plan_csv"])
EXAMPLES  = Path(CFG["paths"]["examples_csv"])
PER_HEAD  = Path(CFG["paths"]["per_head_csv"])
OUTPUT    = Path(CFG["paths"]["output_csv"])

short_df = pd.read_csv(SHORTLIST)   # layer, head, _collapse_score
plan_df  = pd.read_csv(PLAN)        # first col = row_id
ex_df    = pd.read_csv(EXAMPLES)    # row_id, question, ctxs, model_answer, gold_answer, ...

TARGETS: List[Tuple[int,int]] = [(int(r["layer"]), int(r["head"])) for _, r in short_df.iterrows()]

# ------------------------------
# Model & hooks
# ------------------------------
from transformer_lens import HookedTransformer
import torch
import torch.nn.functional as F
from transformer_lens.hook_points import HookPoint

device = "cuda" if torch.cuda.is_available() else "cpu"
model  = HookedTransformer.from_pretrained("gpt2-small", device=device)

# ALPHA_DOSAGE scales selected heads at attn.hook_result:
# 1.0 = no ablation, 0.5 = half-strength, 0.0 = hard-zero (full ablation).
ALPHA_DOSAGE = 0.5

def make_ablation_hooks(targets: List[Tuple[int,int]]):
    """Return TL forward hooks that scale selected heads at attn.hook_result by ALPHA_DOSAGE."""
    by_layer = {}
    for L, H in targets:
        by_layer.setdefault(int(L), []).append(int(H))
    hooks = []
    for L, heads in by_layer.items():
        name = f"blocks.{L}.attn.hook_result"
        def _mk(hlist):
            def _fn(value, hook):
                # value: [batch, heads, seq, d_head]
                value[:, hlist, :, :] = value[:, hlist, :, :] * ALPHA_DOSAGE
                return value
            return _fn
        hooks.append((name, _mk(heads)))
    return hooks

ABLT_HOOKS = make_ablation_hooks(TARGETS)

# ------------------------------
# Collapse metric helpers
# ------------------------------
def attn_entropy(attn_probs: torch.Tensor) -> float:
    """Mean token-wise attention entropy."""
    p = attn_probs.clamp_min(1e-9)
    H = (-p * p.log()).sum(dim=-1)              # [b,h,q]
    return H.mean().item()

def effective_rank(x: torch.Tensor, energy=0.99) -> float:
    """Average effective rank across heads via SVD energy threshold."""
    b,h,s,d = x.shape
    x2 = x.permute(1,0,2,3).contiguous().view(h, b*s, d)   # [h, N, d]
    ranks = []
    for i in range(h):
        u,sing,vh = torch.linalg.svd(x2[i], full_matrices=False)
        cum = (sing**2).cumsum(0) / (sing**2).sum()
        r = int((cum < energy).sum().item()) + 1
        ranks.append(r)
    return float(sum(ranks)/len(ranks))

def self_attn_ratio(attn_probs: torch.Tensor) -> float:
    """Mean diagonal attention mass (self-focus)."""
    b,h,q,k = attn_probs.shape
    diag = torch.arange(q, device=attn_probs.device)
    mass = attn_probs[..., diag, diag]          # [b,h,q]
    return mass.mean().item()

# ------------------------------
# Prompt & generation
# ------------------------------
def build_prompt(row: pd.Series) -> str:
    """Compose a strict copy-from-context prompt; short answers; 'I don't know' if absent."""
    q   = str(row.get("question","")).strip()
    ctx = str(row.get("ctxs","")).strip()
    return (
        "Answer ONLY using the context. If the answer is not explicitly in the context, reply exactly: I don't know.\n"
        "Return a SHORT answer (1–6 words), no extra text.\n\n"
        f"Context:\n{ctx}\n\nQuestion: {q}\nAnswer:"
    )

def greedy_generate(prompt: str, max_new_tokens: int = 16, use_ablation: bool = False,
                    extra_hooks: List[Tuple[str, callable]] = None) -> Tuple[str, dict]:
    """Generate short continuation; optional head ablation; returns text and collected metrics."""
    toks = model.to_tokens(prompt)

    # Collect attention & head-result across (possibly) ablated layers only
    collected_attn = []
    collected_head_out = []
    def attn_hook(value, hook: HookPoint):
        collected_attn.append(value.detach())
        return value
    def result_hook(value, hook: HookPoint):
        collected_head_out.append(value.detach())
        return value

    # choose layers to log (keep cheap)
    target_layers = {L for (L,_) in TARGETS} if use_ablation else {L for (L,_) in TARGETS}
    hooks = []
    for L in sorted(target_layers):
        hooks.append((f"blocks.{L}.attn.hook_attn", attn_hook))
        hooks.append((f"blocks.{L}.attn.hook_result", result_hook))
    if extra_hooks:
        hooks.extend(extra_hooks)

    fwd = ABLT_HOOKS if use_ablation else []
    with model.hooks(fwd_hooks=fwd + hooks):
        out = model.generate(toks, max_new_tokens=max_new_tokens)

    gen = model.to_string(out[0, toks.shape[1]:]).strip()

    # metrics
    if collected_attn:
        attn_cat = torch.cat(collected_attn, dim=0)
        entropy = attn_entropy(attn_cat)
        self_ratio = self_attn_ratio(attn_cat)
    else:
        entropy = float("nan")
        self_ratio = float("nan")
    if collected_head_out:
        head_cat = torch.cat(collected_head_out, dim=0)
        erank = effective_rank(head_cat)
    else:
        erank = float("nan")

    return gen, {"entropy": entropy, "erank": erank, "self_ratio": self_ratio}

# ------------------------------
# Single-example evaluation
# ------------------------------
def evaluate_single_example(row: pd.Series, use_ablation: bool) -> Dict[str, Any]:
    """
    Run one example through GPT-2 small, optionally with head ablation.
    Returns generated answer + collapse metrics (entropy, erank, self_ratio).
    """
    prompt = build_prompt(row)
    answer, mets = greedy_generate(prompt, max_new_tokens=16, use_ablation=use_ablation)
    return {
        "row_id": int(row["row_id"]),
        "hallucinated": 0,         # label comes from external F1 scorer
        "entropy": float(mets["entropy"]),
        "erank":   float(mets["erank"]),
        "hsim":    float("nan"),   # optional, add later
        "self_ratio": float(mets["self_ratio"]),
        "answer": answer
    }

# ------------------------------
# Main paired loop
# ------------------------------
def main():
    plan_id_col = plan_df.columns[0]   # first col is row_id
    plan_ids = set(plan_df[plan_id_col].tolist())
    subset = ex_df[ex_df["row_id"].isin(plan_ids)].copy().sort_values("row_id")

    rows = []
    for _, row in subset.iterrows():
        base  = evaluate_single_example(row, use_ablation=False)
        after = evaluate_single_example(row, use_ablation=True)

        rows.append({
            "row_id": int(row["row_id"]),
            "halluc_before": int(base["hallucinated"]),
            "halluc_after":  int(after["hallucinated"]),
            "d_entropy":    (after["entropy"] - base["entropy"]) if (not pd.isna(base["entropy"]) and not pd.isna(after["entropy"])) else np.nan,
            "d_erank":      (after["erank"]   - base["erank"])   if (not pd.isna(base["erank"])   and not pd.isna(after["erank"]))   else np.nan,
            "d_hsim":       (after["hsim"]    - base["hsim"])    if (not pd.isna(base["hsim"])    and not pd.isna(after["hsim"]))    else np.nan,
            "d_self_ratio": (after["self_ratio"] - base["self_ratio"]) if (not pd.isna(base["self_ratio"]) and not pd.isna(after["self_ratio"])) else np.nan,
            "answer_before": base.get("answer"),
            "answer_after":  after.get("answer")
        })

    out = pd.DataFrame(rows)
    OUTPUT.parent.mkdir(parents=True, exist_ok=True)
    out.to_csv(OUTPUT, index=False)
    print(f"[OK] Saved paired ablation results -> {OUTPUT}  (n={len(out)})")
    print(out[["row_id","answer_before","answer_after"]].head(8).to_string(index=False))

if __name__ == "__main__":
    main()
'''
runner_path = f"{STEP5}/step5_ablation_runner.py"
with open(runner_path,"w") as f:
    f.write(runner_code)
print("Runner written:", runner_path)

# 6) Run the runner from the /workspace path (not /content)
import os
os.chdir(STEP5)
!python step5_ablation_runner.py

# 7) Score with F1 and export extra tables/plots (same logic, with /workspace paths)
import pandas as pd
from scipy.stats import binomtest
import re
from collections import Counter

res_path = paths["output_csv"]
ex_path  = paths["examples_csv"]

res = pd.read_csv(res_path)
ex  = pd.read_csv(ex_path)[["row_id","gold_answer"]]
df  = res.merge(ex, on="row_id", how="left")

def normalize_text(s: str) -> str:
    s = (s or "").lower().strip()
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s)
    return s

def f1_match(pred: str, gold: str) -> float:
    p = normalize_text(pred).split()
    g = normalize_text(gold).split()
    if not p or not g: return 0.0
    from collections import Counter
    pc, gc = Counter(p), Counter(g)
    overlap = sum((pc & gc).values())
    if overlap == 0: return 0.0
    precision = overlap / len(p)
    recall    = overlap / len(g)
    return 2 * precision * recall / (precision + recall + 1e-9)

# hallucination flags (1 = hallucination / F1<0.5)
df["halluc_before_f1"] = df.apply(lambda r: 1 if f1_match(str(r.get("answer_before","")), str(r.get("gold_answer",""))) < 0.5 else 0, axis=1)
df["halluc_after_f1"]  = df.apply(lambda r: 1 if f1_match(str(r.get("answer_after","")),  str(r.get("gold_answer",""))) < 0.5 else 0, axis=1)

before = df["halluc_before_f1"].mean()
after  = df["halluc_after_f1"].mean()
delta  = after - before

b01 = ((df["halluc_before_f1"]==0) & (df["halluc_after_f1"]==1)).sum()  # got worse
b10 = ((df["halluc_before_f1"]==1) & (df["halluc_after_f1"]==0)).sum()  # improved
n   = b01 + b10
pval = binomtest(k=min(b01,b10), n=n, p=0.5, alternative="two-sided").pvalue if n>0 else float("nan")

scored_path = f"{OUT}/step5_ablation_results_scored.csv"
df.to_csv(scored_path, index=False)

print(f"Hallucination (F1<0.5) — before: {before:.3f} | after: {after:.3f} | delta: {delta:.3f}")
print(f"McNemar: 0→1={b01}, 1→0={b10}, pairs={n}, p={pval:.4g}")
print("Saved scored file:", scored_path)
display(df.head(5))

# Build master table (ablation + examples + gold)
m = df.merge(pd.read_csv(paths["examples_csv"]), on="row_id", suffixes=("","_ex"), how="left")

# (Optional) if your QA sheet has metadata columns, join them here
qa = pd.read_csv(paths["qa_csv"])
if "Question" in qa.columns:
    qa["question"] = qa["Question"].astype(str).str.strip()
    keep_cols = ["question"]
    for cand, out_name in [
        ("Interference Type","interference_type"),
        ("Distractor Density","distractor_density"),
        ("Evidence Position","evidence_position"),
        ("Context Length","context_length"),
    ]:
        if cand in qa.columns:
            qa = qa.rename(columns={cand: out_name})
            keep_cols.append(out_name)
    qa_meta = qa[keep_cols].copy()
    if "question" in m.columns:
        m = m.merge(qa_meta, on="question", how="left")

master_path = f"{OUT}/step5_analysis_master.csv"
m.to_csv(master_path, index=False)
print("saved master:", master_path)
print("n=", len(m), "| before=", m["halluc_before_f1"].mean(), "| after=", m["halluc_after_f1"].mean())

# Minimal quick sanity checks on outputs
base = ROOT
check = [
  f"{base}/step5_outputs/step5_head_shortlist.csv",
  f"{base}/step5_outputs/step5_ablation_plan.csv",
  f"{base}/step5/merged_predictions_metrics.csv",
  f"{base}/data/gpt2small_attn_metrics_per_head.csv",
  f"{base}/step5/step5_ablation_config.json",
  f"{base}/step5_outputs/step5_ablation_results.csv",
  f"{base}/step5_outputs/step5_ablation_results_scored.csv",
  f"{base}/step5_outputs/step5_analysis_master.csv",
]
print({p: os.path.exists(p) for p in check})




Torch: 2.1.2+cu121 | CUDA: True
GPU: NVIDIA RTX 2000 Ada Generation
Fri Aug 22 23:57:22 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.57.01              Driver Version: 565.57.01      CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX 2000 Ada Gene...    On  |   00000000:02:00.0 Off |                  Off |
| 30%   26C    P8              7W /   70W |       5MiB /  16380MiB |      0%      Default |
|                                         |                        |    

FileNotFoundError: [Errno 2] No such file or directory: '/workspace/attention-collapse-llm/step5/Distractors - Sheet1.csv'